In [5]:
# importing system modules
import arcpy
import os

In [54]:
# setting workspace
arcpy.env.workspace = "H:/ArcGIS/geogfinalproject/Data"
arcpy.env.overwriteOutput = True

In [6]:
# creating a geodatabase
mygdb = "H:/ArcGIS/geogfinalproject/tri.gdb"

In [56]:
# path for the gdb
dir_path = os.path.dirname(os.path.realpath(mygdb))
outgdb = os.path.basename(mygdb)

In [57]:
# actually creating the GDB
arcpy.CreateFileGDB_management(dir_path, outgdb)

<Result 'H:\\ArcGIS\\geogfinalproject\\tri.gdb'>

In [58]:
# sending communities to shapefile
outworkspace = mygdb
fcList = arcpy.ListFeatureClasses()

In [59]:
# loop to grab the shapefiles
for shapefile in fcList:
    out_featureclass = os.path.join(outworkspace, os.path.splitext(shapefile)[0])

In [60]:
# sending the shapefile to the gdb
arcpy.CopyFeatures_management(shapefile, out_featureclass)

<Result 'H:\\ArcGIS\\geogfinalproject\\tri.gdb\\communityAreas'>

In [61]:
# sending tables to the gdb
tables = arcpy.ListTables()

In [62]:
outLocation = mygdb

In [63]:
arcpy.TableToGeodatabase_conversion(tables, outLocation)

<Result 'H:\\ArcGIS\\geogfinalproject\\tri.gdb'>

In [7]:
# setting new workspace
arcpy.env.workspace = mygdb
arcpy.env.overwriteOutput = True

In [65]:
# creating XY points from TRI release
arcpy.management.XYTableToPoint("fac_release_cleared", "TRIpoints", "long", "lat")

<Result 'H:/ArcGIS/geogfinalproject/tri.gdb\\TRIpoints'>

In [66]:
# adding field for join
arcpy.management.AddField("communityAreas", "areaNUM", "LONG")

<Result 'communityAreas'>

In [68]:
# calculating field to join
arcpy.management.CalculateField("communityAreas", "areaNUM", "!area_num_1!")

<Result 'communityAreas'>

In [69]:
# joining community areas to community data
arcpy.management.AddJoin("communityAreas", "areaNUM", "communityDATA", "commNUM")

<Result 'communityAreas'>

In [70]:
#projecting community areas for spatial join
input_features = r"H:/ArcGIS/geogfinalproject/tri.gdb/communityAreas"
output_feature_class = r"H:/ArcGIS/geogfinalproject/tri.gdb/communityAreas_project"
out_coor_syst = arcpy.SpatialReference('WGS 1984')
arcpy.Project_management(input_features, output_feature_class, out_coor_syst)

<Result 'H:\\ArcGIS\\geogfinalproject\\tri.gdb\\communityAreas_project'>

In [25]:
# joining new projected community to data
arcpy.env.qualifiedFieldNames = False
comFc = "comAreasWithAttributes"
joined_table = arcpy.management.AddJoin("communityAreas_project", "areaNUM", "communityDATA", "commNUM")
# Copy the layer to a new permanent feature class
arcpy.CopyFeatures_management(joined_table, comFc)

<Result 'H:/ArcGIS/geogfinalproject/tri.gdb\\comAreasWithAttributes'>

In [26]:
# fieldmapping to spatially join tri points to community areas
targetFeatures = (comFc)
joinFeatures = ("TRIpoints")

In [27]:
outfc = os.path.join(mygdb, "communityTRI")

In [28]:
# new fieldmappings
fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(targetFeatures)
fieldmappings.addTable(joinFeatures)

In [29]:
# using the total on site field
totalONSITEFieldIndex = fieldmappings.findFieldMapIndex("totalONSITE")
fieldmap = fieldmappings.getFieldMap(totalONSITEFieldIndex)

In [30]:
# getting the output properties
field = fieldmap.outputField

In [31]:
# naming the sum of points
field.name = "sum_onsite"
field.aliasname = "sum_onsite"
fieldmap.outputField = field

In [32]:
# merging
fieldmap.mergeRule = "sum"
fieldmappings.replaceFieldMap(totalONSITEFieldIndex, fieldmap)

In [33]:
# running the spatial join
arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "JOIN_ONE_TO_ONE", "KEEP_ALL", fieldmappings, "WITHIN_A_DISTANCE", "8000 Meters")

<Result 'H:\\ArcGIS\\geogfinalproject\\tri.gdb\\communityTRI'>